In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import yaml
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
sns.set_style('ticks')
sns.set_color_codes()
sns.set_palette('Set1')
sns.set_context('paper')

In [ ]:
colwidth=246 / 72

In [ ]:
with open('../configs/earth_jupiter.yml') as f:
    config = yaml.load(f)

In [ ]:
dirName = '../build/release'
planets = ['Sun', 'Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune',]

posdata = {}
for p in planets:
    try:
        pos = pd.read_csv(os.path.join(dirName, p + '.csv'), header=None)
        if pos.shape[1] == 6:
            pos.columns = ('x', 'y', 'z', 'vx', 'vy', 'vz')
            pos['en'] = pos.vx**2 + pos.vy**2 + pos.vz**2
            pos['rsun'] = np.sqrt(pos.x**2 + pos.y**2 + pos.z**2)
        elif pos.shape[1] == 4:
            pos.columns = ('x', 'y', 'vx', 'vy')
            pos['en'] = pos.vx**2 + pos.vy**2
            pos['rsun'] = np.sqrt(pos.x**2 + pos.y**2)
            
        pos['time'] = pos.index * float(config['timestep'])
        posdata[p] = pos
    except OSError:
        continue

In [ ]:
with sns.color_palette('husl', 9):
    plt.figure(figsize=(colwidth, 0.75*colwidth))
    
    for p in planets:
        try:
            v = posdata[p]
        except KeyError:
            continue
        plt.polar(np.arctan2(v.x, v.y), (np.hypot(v.x, v.y))**(0.3333333333), label=p, linewidth=1)
    plt.gca().set_aspect(1)
    plt.legend(bbox_to_anchor=(1.1, 0.5), loc='center left')
#     plt.savefig('planets_euler.png', bbox_inches='tight', dpi=300)

In [ ]:
with sns.color_palette('husl', 9):
#     plt.figure(figsize=(colwidth, 0.75*colwidth))
    for p in planets:
        try:
            v = posdata[p]
        except KeyError:
            continue
        plt.plot(v.x, np.cos(np.deg2rad(8)) * v.z + np.sin(np.deg2rad(8)) * v.y, label=p, linewidth=1)
    plt.gca().set_aspect(1)
    plt.legend(bbox_to_anchor=(1.1, 0.5), loc='center left')

In [ ]:
%matplotlib notebook
sns.set_context('notebook')

In [ ]:
with sns.color_palette('husl', 9):
    ax = plt.subplot(111, projection='3d')
    for p in planets:
        try:
            v = posdata[p]
        except KeyError:
            continue
        ax.plot(v.x, v.y, v.z, label=p, linewidth=1)
        ax.set_xlim(-50, 50)
        ax.set_ylim(-50, 50)
        ax.set_zlim(-50, 50)
        ax.set_aspect(1)

In [ ]:
plt.figure(figsize=(colwidth*0.8, colwidth*0.8))
epos = posdata['Earth']
jpos = posdata['Jupiter']
plt.plot(epos.x, epos.y, label='Earth')
# plt.plot(jpos.x, jpos.y, label='Jupiter')
plt.plot(0, 0, 'yo', label='Sun')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.gca().set_aspect(1)
plt.xticks(range(-2, 3))
plt.yticks(range(-2, 3))
sns.despine()
plt.grid()
plt.legend(loc='lower center', ncol=2)
plt.xlabel('x [au]')
plt.ylabel('y [au]')

plt.tight_layout()
# plt.savefig('jupiter_1000x.pdf')

In [ ]:
plt.figure(figsize=(colwidth, colwidth * 0.75))
epos = posdata['Earth']
plt.plot(epos.time, epos.x, label='x')
plt.plot(epos.time, epos.y, '--', label='y')
sns.despine()
plt.xlabel('Time [d]')
plt.ylabel('Position [A.U.]')
plt.xticks(365*np.arange(0, 5))
plt.minorticks_on()
plt.grid()
plt.xlim(365*100, 365*105)
plt.legend(loc='lower center', ncol=2)
plt.tight_layout()
# plt.savefig('earthpos_rk4.pdf')

In [ ]:
plt.plot(np.hypot(posdata['Earth'].x, posdata['Earth'].y))

In [ ]:
plt.plot(np.hypot(posdata['Earth'].vx, posdata['Earth'].vy))

In [ ]:
posdata['Earth'].en.mean()

In [ ]:
posdata['Earth'].en.max() - posdata['Earth'].en.min()

In [ ]:
(np.abs(posdata['Earth'].en - posdata['Earth'].en.iloc[0]) / posdata['Earth'].en.iloc[0]).max() * 100

In [ ]:
(np.abs(posdata['Earth'].rsun - posdata['Earth'].rsun.iloc[0]) / posdata['Earth'].rsun.iloc[0]).max() * 100

In [ ]:
plt.figure(figsize=(colwidth, colwidth * 0.75))

plt.plot(posdata['Earth'].time / 365, 
         (posdata['Earth'].en - posdata['Earth'].en.iloc[0]) / posdata['Earth'].en.iloc[0] * 100,
         label='Kinetic energy')
plt.plot(posdata['Earth'].time / 365, 
         (posdata['Earth'].rsun - posdata['Earth'].rsun.iloc[0]) / posdata['Earth'].rsun.iloc[0] * 100, '--',
         label='Earth-Sun dist.')
sns.despine()
plt.xlabel('Time [y]')
plt.ylabel('Deviation from initial value [%]')
plt.legend(loc=8, ncol=2)
plt.ylim(-3.5, 3.5)
plt.tight_layout()
plt.grid()
# plt.savefig('jupiter_devs_1x.pdf')

In [ ]:
epos = posdata['Earth']
aphs = []
peris = []
for yr in range(0, 100):
    dmin, dmax = yr*365, (yr + 1)*365
    aphs.append(epos.iloc[dmin:dmax].rsun.max())
    peris.append(epos.iloc[dmin:dmax].rsun.min())
    
plt.plot(aphs)
plt.plot(peris)

In [ ]:
plt.figure(figsize=(colwidth, colwidth * 0.75))
plt.plot(posdata['Earth'].time / 365, posdata['Earth'].rsun)
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
sns.despine()
plt.xlabel('Time [y]')
plt.ylabel('Earth-Sun distance [au]')
plt.grid()

In [ ]:
plt.plot(posdata['Earth'].rsun / posdata['Earth'].rsun.iloc[0] - 1)
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)